# Descripción del proyecto
La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

Se tiene acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos. Se tiene que  crear un modelo que escoja el plan correcto con la mayor exactitud posible

# Segmentacion de nuestros datos 

In [1]:
# Importando las librerias necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

df = pd.read_csv('/datasets/users_behavior.csv')
print(df.info())
display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [2]:
# dividiendo mi conjunto de datos 
# 20% a test 
df_temp, df_test = train_test_split(df, test_size = 0.20, random_state = 54321) 
# 60% a train y %20 a valid porque el 25% de 80% es 20%
df_train, df_valid = train_test_split(df_temp, test_size = 0.25, random_state = 54321)

In [3]:
# Datos para entrenamiento 
features_train = df_train.drop(['is_ultra'], axis = 1)
target_train = df_train['is_ultra']

# Datos para validacion
features_valid = df_valid.drop(['is_ultra'], axis = 1)
target_valid = df_valid['is_ultra']

# Datos para test
features_test = df_test.drop(['is_ultra'], axis = 1)
target_test = df_test['is_ultra']

# Evaluando la calidad de distintos modelos 

## Modelos de clasificación 

### Arbol de decisión 

In [4]:
from sklearn.tree import DecisionTreeClassifier

best_score = 0
best_depth = 0
for depth in range(1,11):
    model = DecisionTreeClassifier(random_state = 54321, max_depth = depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    acurracy_decisionTree = model.score(features_valid, target_valid)
    if acurracy_decisionTree > best_score:
        best_score = acurracy_decisionTree
        best_depth = depth
print("La exactitud del modelo en el conjunto de validación (n_estimators = {}): {}".format(best_depth, best_score))      


La exactitud del modelo en el conjunto de validación (n_estimators = 5): 0.8180404354587869


Para garantizar que la evaluación en el conjunto de prueba refleje el rendimiento del modelo con el mejor hiperparámetro y aprovechando toda la información válida, se reentrena el modelo final justo antes de medir su exactitud en test.

In [5]:
# Entrenando el modelo nuevamente con los hiperparametros obtenidos
modelDecisionTree = DecisionTreeClassifier(random_state = 54321, max_depth = best_depth)
modelDecisionTree.fit(features_train, target_train)
scoreDecisionTree =  modelDecisionTree.score(features_test, target_test)
print("La exactitud del modelo en el conjunto de test = {}".format(scoreDecisionTree))



La exactitud del modelo en el conjunto de test = 0.7698289269051322


### Bosque aleatorio 

In [6]:
from sklearn.ensemble import RandomForestClassifier

best_score = 0
best_est = 0
for estimator in range(1,11):
    model = RandomForestClassifier(random_state = 54321, n_estimators = estimator)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    acurracy_randomForest = model.score(features_valid, target_valid)
    if acurracy_randomForest > best_score:
        best_score = acurracy_randomForest
        best_est = estimator

print("La exactitud del mejor modelo en el conjunto de validación (n_estimators = {}): {}".format(best_est, best_score))

La exactitud del mejor modelo en el conjunto de validación (n_estimators = 10): 0.8258164852255054


Como en el caso del arbol de decisión se tiene que reentrenar el modelo 

In [7]:
# Entrenando el modelo nuevamente con lo obtenido
modelForestClassifier = RandomForestClassifier(random_state = 54321, n_estimators = best_est)
modelForestClassifier.fit(features_train, target_train)
scoreRandomForest = modelForestClassifier.score(features_test, target_test)
print("La exactitud del modelo en el conjunto de test = {}".format(scoreRandomForest))

La exactitud del modelo en el conjunto de test = 0.7838258164852255


### Regresión logística

In [8]:
from sklearn.linear_model import LogisticRegression


modelLogisticRegression = LogisticRegression(random_state=54321, solver = 'liblinear')
modelLogisticRegression.fit(features_train, target_train)
scorelRegression = modelLogisticRegression.score(features_test, target_test) # No usamos el conjunto de validación pues no estamos ajustando hiperparametros
print("Accuracy del modelo de regresión logística en el conjunto de test:", scorelRegression)

Accuracy del modelo de regresión logística en el conjunto de test: 0.6780715396578538


# Pruebas de cordura

Podemos realizar una prueba de cordura con un dataset que sabremos siempre su respuesta

In [9]:
datasetCordura = pd.DataFrame([
  # Ningún uso
  {'calls':  0,   'minutes':    0,   'messages':   0,   'mb_used':     0},
  # Uso muy alto
  {'calls':200, 'minutes': 2000, 'messages': 500, 'mb_used':100_000},
  # Uso promedio 
  {'calls':20,  'minutes':  200,  'messages':  30,  'mb_used':  1000},
])


## Predicciones

In [11]:
preds = modelDecisionTree.predict(datasetCordura)
print("Predicciones arbol de decision:", preds)
preds = modelForestClassifier.predict(datasetCordura)
print("Predicciones bosque aleatorio:", preds)
preds = modelLogisticRegression.predict(datasetCordura)
print("Predicciones regresión lógistica:", preds)

Predicciones arbol de decision: [0 1 0]
Predicciones bosque aleatorio: [1 1 1]
Predicciones regresión lógistica: [0 0 1]


Al evaluar los tres modelos sobre escenarios “obvios” observamos:

- Árbol de decisión: predice [0, 1, 0], acertando en los dos extremos y ofreciendo un resultado coherente en el caso intermedio.

- Bosque aleatorio: predice [1, 1, 1], recomendando Ultra en todos los casos, lo cual no tiene sentido para el usuario sin consumo (primer caso).

- Regresión logística: predice [0, 0, 1], falla en el extremo (debería sugerir Ultra) y muestra un comportamiento menos consistente.

Por tanto, solo el árbol de decisión pasa completamente la prueba de cordura, reaccionando de forma intuitiva a los escenarios definidos.

# Eligiendo al mejor modelo 

In [12]:
def bestModel(scoreDecisionTree, scoreRandomForest, scorelRegression):
    if scoreDecisionTree > scoreRandomForest:
        if scoreDecisionTree > scorelRegression:
            return print("El modelo con mayor precision es el arbol de decision con {} de precision".format(scoreDecisionTree))
    if scoreRandomForest > scorelRegression:
        return print("El modelo con mayor precision es el bosque de decision con {} de precision".format(scoreRandomForest))
    return print("El modelo con mayor precision es la regresión logística con {} de precision".format(scorelRegression))

bestModel(scoreDecisionTree, scoreRandomForest, scorelRegression)


El modelo con mayor precision es el bosque de decision con 0.7838258164852255 de precision


# Conclusión

El árbol de decisión, con un 76,98 % de exactitud en el conjunto de prueba y un comportamiento coherente en los escenarios de cordura, supera el umbral mínimo del 75 % exigido y demuestra una capacidad intuitiva para recomendar planes según el perfil de uso; aunque el bosque aleatorio logró una precisión ligeramente mayor (78,38 %), su tendencia a asignar Ultra en todos los casos triviales evidencia un sesgo que lo hace menos confiable para usuarios de bajo consumo, y la regresión logística quedó por debajo del umbral (67,81 %), por lo que el árbol de decisión resulta ser la opción óptima para la recomendación de planes Smart o Ultra en Megaline.